# Summary of this notebook
### Clean the power data:
- For us training dataset is the data from the years 2019 through 2022. The testing dataset is the data from 2023.
- The power data contains 5 duplicates - one from each year. This happened due to daylight saving time zone shifts. The duplicates in years 2019 through 2022 are close to each other, so we just manually changed one of the `01:00:00` entry to `00:00:00` entry based on the whichever is closed to the previous day `23:00:00` value.
- For the year 2023, the day `03-12` has 25 entries and `11-05` has 23 entries, so the missing entry is roughly 8 months away from the missing entry. We have the following options:
    - Remove both these two days from the testing data.
    - Shift all the entries from `03-12` till `11-05` by one hour. Note: this is equivalent to treating "hour 1", "hour 2", and so on for a year, because one of the duplicate would become entry `hour n` and the other one would become `hour n+1` basically shifting everything by one hour after the duplicate till the missing.
    - Pick one of the duplicate on `03-12` and remove the other. Then fill the missing entry on `11-05` with average of the nearest two.
- Once a consensus is reached on which of the following option we choose, we will create the `main_testing_dataframe.csv`. I commented these codelines for now.

### Merge Weather files (in service):
- Pretty straight-forward as there is no time-zone issue. All times are already in a "time-zone independent" format in the original datasets.
- Just merged them into the file `final_weather_only.csv`

### Merge both weather and power:
- Merged `weather_training_df`, and `power_training_df` into the file `main_training_dataframe.csv`.
- Commented out the codelines of `main_testing_dataframe.csv` for now. This would be the merged dataframes `weather_testing_df`, and `power_testing_df`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Clean the power data

In [2]:
#Power output HQ dataset (with only wind powercolumn)
power_df = pd.read_csv("../raw_data/historique-production-electricite-quebec.csv")[["Date","Wind"]]

There are some duplicate dates due to time-zone related issue. I am printing them here.

In [3]:
#Checking duplicate 'Date' in the original dataset
power_df_strtime = power_df
print("Duplicated places:")
print(power_df_strtime[power_df_strtime['Date'].duplicated()])

Duplicated places:
                            Date     Wind
5476   2020-11-01T01:30:00-05:00  2737.00
14385  2021-11-07T01:30:00-05:00  2584.00
22988  2019-11-03T01:30:00-05:00   686.00
28901  2022-11-06T01:30:00-05:00  2576.00
41485  2023-03-12T03:30:00-04:00   142.97


Convert all time strings to pandas Timestamp objects with UTC flag true.

In [4]:
#Format time from string to TimeStamp and converted all times to UTC
power_df['Date'] = pd.to_datetime(power_df['Date'], utc =True)

But this shifts some datapoints from Year N to Year N+1, so we shift everything by 5 hours so that the first hour is 2019-01-01 00:00:00 and the last hour is 2023-12-31 23:00:00.

In [5]:
power_df['Date'] = power_df['Date'].apply(lambda x: x - pd.Timedelta(hours=5))

In [6]:
#power_df['TimeZone'] = power_df['Date'].apply(lambda x: x.utcoffset())
#print(power_df['TimeZone'].unique())

After this, we can treat our data time-zone independent. We we remove the time-zone information all together.

In [6]:
power_df['Date'] = power_df['Date'].dt.tz_localize(None)#remove time-zone information

In [7]:
power_df.head()

,Date,Wind
0,2023-01-01 00:30:00,151.07
1,2023-01-01 05:30:00,679.07
2,2023-01-01 06:30:00,829.70
3,2023-01-01 08:30:00,863.91
4,2023-01-01 11:30:00,639.35


In [8]:
#Sort the dataset by date
power_df = power_df.sort_values('Date')

#Replace 'Date' column in the format Year:MonthDay:Hour:00 for consistant formating with weather data
power_df['Date'] = power_df['Date'].apply(lambda x: x.floor('h'))

#Adding year and month/day in the dataset
power_df['Year'] = power_df['Date'].apply(lambda x: x.year)                     #adding a year column
power_df['MonthDay'] = power_df['Date'].apply(lambda x: x.strftime('%m-%d'))    #adding a Month/Day column

Renaming the `Date` column to `time` to be consistent with the weather files.

In [9]:
power_df = power_df.rename(columns={'Date' : 'time'})

## Ignore this part
I just tried to check the missing timestamps and the time-zone related issues.

In [10]:
timestamps = pd.date_range(start="2019-01-01 00:00:00", end="2023-12-31 23:00:00", freq="h")

In [11]:
# Step 3: Find missing timestamps
missing = timestamps.difference(power_df['time'])
print("Missing timestamps:")
print(missing, len(missing))

# Step 4: Find duplicate timestamps
duplicates = power_df['time'][power_df['time'].duplicated(keep=False)]
print("Duplicate timestamps:")
print(duplicates, len(duplicates))

Missing timestamps:
DatetimeIndex(['2019-11-03', '2020-11-01', '2021-11-07', '2022-11-06',
               '2023-11-05'],
              dtype='datetime64[ns]', freq=None) 5
Duplicate timestamps:
22988   2019-11-03 01:00:00
17194   2019-11-03 01:00:00
5475    2020-11-01 01:00:00
5476    2020-11-01 01:00:00
14385   2021-11-07 01:00:00
14383   2021-11-07 01:00:00
28901   2022-11-06 01:00:00
11380   2022-11-06 01:00:00
589     2023-03-12 02:00:00
41485   2023-03-12 02:00:00
Name: time, dtype: datetime64[ns] 10


In [13]:
#print(len(power_df[(power_df['Date'] >= pd.Timestamp('2023-01-01 00:00:00') - pd.Timedelta(hours=12)) & 
#                (power_df['Date'] <= pd.Timestamp('2023-03-12 12:00:00') + pd.Timedelta(hours=12))]))

In [14]:
#print(len(power_df[(power_df['Date'] >= pd.Timestamp('2020-03-08 00:00:00')) & 
#               (power_df['Date'] <= pd.Timestamp('2020-03-08 23:00:00'))]))

In [15]:
#for year in range(2019, 2024):
#    print(len(power_df[power_df['Year'].astype(int) == year]))

## Duplicate handling

In [12]:
dupes_df = power_df[power_df['time'].duplicated(keep=False)]
dupes_df_sorted = dupes_df.sort_values('time')

for ts in dupes_df_sorted['time'].unique():
    print(f"\nNearby rows for duplicated timestamp: {ts}")
    nearby = power_df[(power_df['time'] >= ts - pd.Timedelta(hours=2)) & 
                (power_df['time'] <= ts + pd.Timedelta(hours=1))]
    print(nearby.sort_values('time'))


Nearby rows for duplicated timestamp: 2019-11-03 01:00:00
                     time   Wind  Year MonthDay
17193 2019-11-02 23:00:00  773.0  2019    11-02
22988 2019-11-03 01:00:00  686.0  2019    11-03
17194 2019-11-03 01:00:00  712.0  2019    11-03
22989 2019-11-03 02:00:00  702.0  2019    11-03

Nearby rows for duplicated timestamp: 2020-11-01 01:00:00
                    time    Wind  Year MonthDay
5474 2020-10-31 23:00:00  2459.0  2020    10-31
5475 2020-11-01 01:00:00  2212.0  2020    11-01
5476 2020-11-01 01:00:00  2737.0  2020    11-01
5477 2020-11-01 02:00:00  2723.0  2020    11-01

Nearby rows for duplicated timestamp: 2021-11-07 01:00:00
                     time    Wind  Year MonthDay
31895 2021-11-06 23:00:00  2819.0  2021    11-06
14385 2021-11-07 01:00:00  2584.0  2021    11-07
14383 2021-11-07 01:00:00  2689.0  2021    11-07
31896 2021-11-07 02:00:00  2054.0  2021    11-07

Nearby rows for duplicated timestamp: 2022-11-06 01:00:00
                     time    Wind  Year

### Manually changed the 4 duplicates in 2019-2022 from 01:00:00 to 00:00:00 and chose the closer value to 23:00:00 of previous day.


In [13]:
power_df.at[17194, 'time'] = pd.Timestamp('2019-11-03 00:00:00')
power_df.at[5475, 'time'] = pd.Timestamp('2020-11-01 00:00:00')
power_df.at[14383, 'time'] = pd.Timestamp('2021-11-07 00:00:00')
power_df.at[11380, 'time'] = pd.Timestamp('2022-11-06 00:00:00')

The only duplicate present is in the year 2023, which we'll handle later.

In [14]:
power_df[power_df['time'].duplicated()]

,time,Wind,Year,MonthDay
41485,2023-03-12 02:00:00,142.97,2023,03-12


### Save the modifies power file with years 2019 through 2022
Save this power dataframe in file `../final_dataframes/training data_final_power_only.csv`

In [15]:
power_training_df = power_df[power_df['time'].dt.year.astype(str) != '2023']

In [16]:
power_training_df['time'].dt.year.unique()

array([2019, 2020, 2021, 2022], dtype=int32)

In [17]:
# Save the dataset
power_training_df.to_csv('../final_dataframes/final_training_data_power_only.csv', index=False)

### WARNING for this section: this is the data for 2023 with the duplicate
We'll deal with this once we agree on a criteria specifically for the year 2023. We'll save the dataframe letter in a csv.

In [ ]:
#power_testing_df = power_df[power_df['time'].dt.year.astype(str) == '2023']

In [ ]:
#Optional: Save power_testing_df
#power_testing_df.to_csv('../final_dataframes/final_testing_data_power_only.csv', index=False)

# Weather files

In [18]:
#File names for weather data of different farms
weather_files = os.listdir('../windfarm_weather_data_in_service')

### Just checking the first farm to see how the data look

In [19]:
farm_wdf = pd.read_csv(f"../windfarm_weather_data_in_service/{weather_files[0]}")
farm_wdf['time'] = pd.to_datetime(farm_wdf['time'])                  #convert time to Timestamp object
farm_wdf = farm_wdf.sort_values('time')                                         #sort values using time

#farm_wdf['Year'] = farm_wdf['time'].apply(lambda x: x.year)                     #adding a year column
#Remove data from 2024 as we don't have power data from 2024
farm_wdf = farm_wdf[farm_wdf['time'].dt.year.astype(str) != '2024']

In [20]:
farm_wdf

,Unnamed: 0,time,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,location_1
0,0,2019-01-01 00:00:00,-7.4,81,17.9,146,Baie-des-Sables wind farm
1,1,2019-01-01 01:00:00,-7.0,80,18.9,145,Baie-des-Sables wind farm
2,2,2019-01-01 02:00:00,-7.1,80,21.3,146,Baie-des-Sables wind farm
3,3,2019-01-01 03:00:00,-6.8,80,22.2,148,Baie-des-Sables wind farm
4,4,2019-01-01 04:00:00,-6.8,81,22.4,143,Baie-des-Sables wind farm
...,...,...,...,...,...,...,...
43819,43819,2023-12-31 19:00:00,-6.0,66,16.2,302,Baie-des-Sables wind farm
43820,43820,2023-12-31 20:00:00,-6.2,65,15.2,306,Baie-des-Sables wind farm
43821,43821,2023-12-31 21:00:00,-6.2,65,15.0,312,Baie-des-Sables wind farm
43822,43822,2023-12-31 22:00:00,-6.4,66,15.3,317,Baie-des-Sables wind farm


### Using some part of Desmond code to merge

In [ ]:
merge_wdf = pd.DataFrame()             #create a new pandas dataframe to store the merge weather files
for filename in weather_files:
    try: 
        df = pd.read_csv(f"../windfarm_weather_data_in_service/{filename}") #read each file
        df['time'] = pd.to_datetime(df['time'])                      #convert time to pandas timestamp
        df = df.sort_values('time')                                  #sort values using time
        df = df[df['time'].dt.year.astype(str) != '2024']            #remove entries in 2024
        if filename == weather_files[0]:                             #store the first dataframe   
            merge_wdf = df
        else:
            #merge the other dataframes using these columns
            merge_wdf = pd.merge(merge_wdf, df, on=['Unnamed: 0', 'time'])  
    except:
            print(filename)

No need to have those location names in the dataframe as we already know the labels, so let's remove them.

In [22]:
locations = [f"location_{i}" for i in range(1, 40)]
merge_wdf = merge_wdf.drop(columns=locations, axis=1)

Save this weather dataframe in file `../final_dataframes/final_weather_only.csv`

In [23]:
merge_wdf.to_csv('../final_dataframes/final_weather_only.csv', index=False)

# Merge both power and weather files for training

In [24]:
#split the weather into training and testing
weather_training_df = merge_wdf[merge_wdf['time'].dt.year.astype(str) != '2023']
weather_testing_df = merge_wdf[merge_wdf['time'].dt.year.astype(str) == '2023']

In [31]:
#Optional: Save weather_training_df and weather_testing_df
#weather_training_df.to_csv('../final_dataframes/final_training_data_weather_only.csv', index=False)
#weather_testing_df.to_csv('../final_dataframes/final_testing_data_weather_only.csv', index=False)

In [25]:
main_training_df = pd.merge(weather_training_df, power_training_df, on=['time'])

### Finally save the main dataframe
Our final dataframe in location `../final_dataframes/main_training_dataframe.csv`.

In [26]:
main_training_df.to_csv('../final_dataframes/main_training_dataframe.csv', index=False)

## Once power testing consensus is reached, we can merge weather_testing_df with power_testing_df

In [ ]:
#main_testing_df = pd.merge(weather_testing_df, power_testing_df, on=['time'])